In [1]:
suppressPackageStartupMessages(library(DESeq2))

In [2]:
table_dir <- '/home/niklas/projects/niche_environments_FIBROSIS/PCLS_human/01_data/ASK_joint/'

In [3]:
counts <- read.csv(paste0(table_dir, '211201_PCLS_human_ASK_joint_SyntheticBulk.csv'))
metadata <-  read.csv(paste0(table_dir, 'metadata.csv'), sep = ';')

In [4]:
metadata

identifier,patient,name,modality,treatment,timepoint,condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
muc24271,ASK591,ASK591_PCLS_FC,PCLS,FC,day 6,Fibrotic Cocktail
muc24272,ASK591,ASK591_PCLS_CC,PCLS,CC,day 6,Control Cocktail
muc24273,ASK591,ASK591_PCLS_FC+Nintedanib,PCLS,FC+Nintedanib,day 6,Fibrotic Cocktail + Treatment
muc24274,ASK591,ASK591_PCLS_FC+Pirfenidone,PCLS,FC+Pirfenidone,day 6,Fibrotic Cocktail + Treatment
muc24275,ASK591,ASK591_PCLS_FC+CMP4,PCLS,FC+CMP4,day 6,Fibrotic Cocktail + Treatment
muc24276,ASK591,ASK591_Tissue,Tissue,none,day 0,none
muc24279,ASK591,ASK591_PCLS_FC,PCLS,FC,day 6,Fibrotic Cocktail
muc31175,ASK627,ASK627_PCLS_FC,PCLS,FC,day 6,Fibrotic Cocktail
muc31176,ASK627,ASK627_PCLS_FC+Nintedanib,PCLS,FC+Nintedanib,day 6,Fibrotic Cocktail + Treatment


In [5]:
head(counts,5)

,Genes,muc24271,muc24272,muc24273,muc24275,muc24279,muc31175,muc31176,muc31177,muc31178,muc31185
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,OR4F5,0,0,0,0,0,0,0,0,0,0
2,OR4F29,0,0,0,0,0,0,0,0,0,0
3,OR4F16,0,0,0,0,0,0,0,0,0,0
4,SAMD11,475,86,283,279,353,1287,761,2390,1891,903
5,NOC2L,1804,2035,2769,3006,1433,3351,4246,3471,3729,3960


In [6]:
rownames(counts) <- counts$Genes
counts$Genes <- NULL

In [7]:
counts <- counts[which(colnames(counts) %in% c('muc24271','muc24272','muc31179','muc31185','muc31175'))]
head(counts,5)

,muc24271,muc24272,muc31175,muc31185
,<int>,<int>,<int>,<int>
OR4F5,0,0,0,0
OR4F29,0,0,0,0
OR4F16,0,0,0,0
SAMD11,475,86,1287,903
NOC2L,1804,2035,3351,3960


In [9]:
metadata <- metadata[which(metadata$identifier %in% colnames(counts)), ]
metadata

,identifier,patient,name,modality,treatment,timepoint,condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,muc24271,ASK591,ASK591_PCLS_FC,PCLS,FC,day 6,Fibrotic Cocktail
2,muc24272,ASK591,ASK591_PCLS_CC,PCLS,CC,day 6,Control Cocktail
8,muc31175,ASK627,ASK627_PCLS_FC,PCLS,FC,day 6,Fibrotic Cocktail
13,muc31185,ASK627,ASK627_PCLS_CC,PCLS,CC,day 6,Control Cocktail


In [10]:
dds <- DESeqDataSetFromMatrix(countData = counts,
                              colData = metadata,
                              design = ~ treatment)
dds

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


class: DESeqDataSet 
dim: 20613 4 
metadata(1): version
assays(1): counts
rownames(20613): OR4F5 OR4F29 ... AC213203.4 AC213203.1
rowData names(0):
colnames(4): muc24271 muc24272 muc31175 muc31185
colData names(7): identifier patient ... timepoint condition

In [12]:
fpkm(dds, robust=FALSE)

ERROR: Error in fpkm(dds, robust = FALSE): rowRanges(object) has all ranges of zero width.
the user should instead supply a column, mcols(object)$basepairs,
which will be used to produce FPKM values


In [10]:
## filtering - keep only genes which have 10 total counts across all conditons
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

In [11]:
## set reference level
dds$treatment <- relevel(dds$treatment, ref = 'CC')

In [12]:
dds <- DESeq(dds)
res <- results(dds)
res

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



log2 fold change (MLE): treatment FC vs CC 
Wald test p-value: treatment FC vs CC 
DataFrame with 17020 rows and 6 columns
            baseMean log2FoldChange     lfcSE      stat    pvalue      padj
           <numeric>      <numeric> <numeric> <numeric> <numeric> <numeric>
SAMD11      590.1674       1.039719  0.805763  1.290353  0.196928  0.999984
NOC2L      2638.1519      -0.154996  0.325167 -0.476664  0.633602  0.999984
KLHL17      450.3547       0.712674  0.468922  1.519814  0.128558  0.999984
PLEKHN1     509.2745       0.810455  0.493613  1.641885  0.100614  0.999984
PERM1        13.6569       0.194192  1.478496  0.131344  0.895503  0.999984
...              ...            ...       ...       ...       ...       ...
AL592183.1 561.14592     -0.4093135  0.802147 -0.510273  0.609860  0.999984
AC007325.4 150.70771      0.2398174  0.541482  0.442890  0.657845  0.999984
AL354822.1 408.67842      0.1639664  0.539840  0.303731  0.761332  0.999984
AC233755.2   1.71893      1.8645797  4.75

In [13]:
res <- data.frame(res)
head(res,5)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SAMD11,590.16742,1.0397188,0.8057631,1.2903529,0.1969282,0.9999845
NOC2L,2638.15190,-0.1549955,0.3251675,-0.4766637,0.6336016,0.9999845
KLHL17,450.35467,0.7126741,0.4689219,1.5198143,0.1285577,0.9999845
PLEKHN1,509.27453,0.8104553,0.4936126,1.6418855,0.1006137,0.9999845
PERM1,13.65693,0.1941921,1.4784957,0.1313444,0.8955029,0.9999845


In [14]:
write.csv(res, paste0(table_dir, '211207_ASK_joint_synthetic_bulk_DGE.csv'))